In [2]:
import datetime
import shutil
import time
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
from PIL import Image
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import (resnet18, resnet34, resnet50, resnet101,
                                resnet152, resnext50_32x4d, resnext101_32x8d,
                                wide_resnet50_2, wide_resnet101_2)
from tqdm import tqdm

In [3]:
class weighted_MSELoss(nn.Module):

    def forward(self, inputs, targets):
        return (targets * (inputs - targets)**2).mean()


In [4]:
# Hyperparameters
initial_lr = 0.005
num_epochs = 50
batch_size = 64

# Data
train_img_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\train')
test_img_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\test')

device = torch.device("cuda:0")

In [4]:
class Trainer:

    def __init__(self, model) -> None:
        self.model = model.to(device)
        self.optimizer = Adam(self.model.parameters(), lr=initial_lr)
        self.loss_func = weighted_MSELoss().cuda()
        self.scheduler = StepLR(self.optimizer, step_size=7, gamma=0.4)

        self.total_time = 0
        self.train_loader = None
        self.test_loader = None
        self.labels = None

        self.create_output_dir()
        self.load_data()

    def create_output_dir(self):
        # 获取当前时间的字符串表示形式
        current_time = datetime.datetime.now().strftime(f"%Y-%m-%d_%H-%M-%S")
        # 生成以当前时间命名的目录路径
        self.output_dir = Path.cwd().parent / 'output' / 'run' / current_time
        # 创建目录
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def load_data(self):
        # 加载训练集
        transform_train = transforms.Compose([
            transforms.Grayscale(1),
            transforms.RandomHorizontalFlip(),  # 随机水平翻转
            transforms.RandomVerticalFlip(),  # 随机垂直翻转
            transforms.RandomRotation(360),  # 随机旋转
            transforms.ToTensor()
        ])
        train_data = datasets.ImageFolder(root=train_img_path.as_posix(), transform=transform_train)
        self.train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)

        # 加载测试集
        transform_test = transforms.Compose([transforms.Grayscale(1), transforms.ToTensor()])
        test_data = datasets.ImageFolder(root=test_img_path.as_posix(), transform=transform_test)
        self.test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, pin_memory=True)

        self.labels = [x for x, _ in sorted(test_data.class_to_idx.items(), key=lambda x: x[1])]

    def train(self):
        self.model.train()
        running_loss = 0
        for inputs, labels in self.train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float().view(-1, 1) + 1
            outputs = self.model(inputs) + 1
            self.optimizer.zero_grad()
            loss = self.loss_func(outputs, labels)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        self.scheduler.step()
        return running_loss / len(self.train_loader.dataset)

    def validate(self):
        self.model.eval()
        running_loss = 0
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().view(-1, 1) + 1
                outputs = self.model(inputs) + 1
                loss = self.loss_func(outputs, labels)
                running_loss += loss.item() * inputs.size(0)

        return running_loss / len(self.test_loader.dataset)

    def run(self):
        with open(self.output_dir / 'log.txt', 'w') as log:
            log.write(f'Number of epochs: {num_epochs}\n')
            log.write(f'Initial learning rate: {initial_lr}\n')
            log.write(f'Batch size: {batch_size}\n')
            log.write(f'Optimizer: Adam\n')
            log.write(f'Scheduler: StepLR\n')
            log.write(f'Loss function: MSELoss\n')
            log.write(f'Model: resnext50_32x4d\n')
            log.write(f'Device: {device}\n\n')

            with tqdm(range(num_epochs), desc='Progress') as tbar:
                for epoch in range(num_epochs):
                    epoch_start = time.time()
                    train_loss = self.train()
                    valid_loss = self.validate()
                    time_elapsed = time.time() - epoch_start

                    log.write(f'Epoch {epoch+1:02d}/{num_epochs}, ')
                    log.write(f'Training Loss: {train_loss:.4f}, ')
                    log.write(f'Validation Loss: {valid_loss:.4f}, ')
                    log.write(f'Time: {time_elapsed:.2f}s\n')
                    log.flush()
                    tbar.update()

            torch.save(self.model, self.output_dir / f'model.pth')
            overall_acc, average_loss = self.eval_accuracy()
            log.write(f'Overall accuracy: {overall_acc:.4f}%\n')
            log.write(f'Average loss: {average_loss:.4f}\n')

    def eval_accuracy(self):
        # 精度评估
        difference = []
        weight = []
        losses = []

        self.model.eval()
        with torch.no_grad():
            for inputs, labels in self.test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().view(-1, 1) + 1
                outputs = self.model(inputs) + 1
                # 四舍五入到最近的整数, 将所有小于1的值设为1
                outputs = outputs.round().clamp(min=1)

                difference.append((outputs - labels).abs().mean())
                losses.append(self.loss_func(outputs, labels))
                weight.append(inputs.size(0))

        # 计算精度
        overall_acc = np.average(difference)
        average_loss = np.average(losses, weights=weight)
        return overall_acc, average_loss

    def predict(self):
        pred_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\pred\pred')
        dst_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\output')

        self.model.eval()
        with torch.no_grad():
            for pic_path in pred_path.iterdir():
                pic = Image.open(str(pic_path))
                pic_tensor = torch.from_numpy(np.array(pic, dtype=np.float32)) / 255

                input = pic_tensor.unsqueeze(0).unsqueeze(0).to(device)
                output = round(self.model(input).item()) + 1
                
                pic.close()
                dst_path = Path(r'C:\Users\xianyu\GraduationProject\UAV_YUNNAN_DATA\last_labels\output')
                dst_path = dst_path / f'{output}_{pic_path.name}'

                shutil.copy(pic_path, dst_path)

In [5]:
# 因为任务是回归，所以将最后一层的输出维度设置为1
model = resnext50_32x4d(weights=None, num_classes=1)
# 重置第一层卷积层，将输入由三通道变为单通道
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

trainer = Trainer(model)
trainer.run()

Progress: 100%|██████████| 50/50 [26:52<00:00, 32.24s/it]


In [18]:
a=torch.linspace(0,10,20,dtype=torch.float32)
a

tensor([ 0.0000,  0.5263,  1.0526,  1.5789,  2.1053,  2.6316,  3.1579,  3.6842,
         4.2105,  4.7368,  5.2632,  5.7895,  6.3158,  6.8421,  7.3684,  7.8947,
         8.4211,  8.9474,  9.4737, 10.0000])